In [1]:
import pandas as pd
import numpy as np
from geojson import Point
from keplergl import KeplerGl
import requests
import pprint
from bson.son import SON
from pymongo import MongoClient

# Ciudades 

* Londres: [51.50853,-0.12574]
* Nueva York:  [40.71427,-74.00597]
* Los Angeles: [34.05223,-118.24368]
* Tokyo: [35.6895,139.69171]
* Washington D.C.: [38.89511,-77.03637]

In [2]:
cities = [[51.50853,-0.12574],[40.71427,-74.00597],[34.05223,-118.24368],[35.6895,139.69171],[38.89511,-77.03637]]

## Airports

In [3]:
airports = pd.read_csv("input/airports.csv")

In [4]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7698 entries, 0 to 7697
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AirportID  7698 non-null   int64  
 1   Name       7698 non-null   object 
 2   City       7649 non-null   object 
 3   Country    7698 non-null   object 
 4   IATA       7698 non-null   object 
 5   ICAO       7698 non-null   object 
 6   Latitude   7698 non-null   float64
 7   Longitude  7698 non-null   float64
 8   Altitude   7698 non-null   int64  
 9   Timezone   7698 non-null   object 
 10  DST        7698 non-null   object 
 11  Tz         7698 non-null   object 
 12  Type       7698 non-null   object 
 13  Source     7698 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 842.1+ KB


In [5]:
airports.dropna(inplace=True)
airports["location"] = ''

In [6]:
for index, row in airports.iterrows():
    airports["location"][index] = Point((row['Longitude'], row['Latitude']))

<ipython-input-6-cc2f327561d8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports["location"][index] = Point((row['Longitude'], row['Latitude']))


In [7]:
airports

,AirportID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz,Type,Source,location
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,"{'type': 'Point', 'coordinates': [145.391998, ..."
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,"{'type': 'Point', 'coordinates': [145.789001, ..."
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,"{'type': 'Point', 'coordinates': [144.296005, ..."
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,"{'type': 'Point', 'coordinates': [146.725977, ..."
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,"{'type': 'Point', 'coordinates': [147.220001, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports,"{'type': 'Point', 'coordinates': [52.478298, 7..."
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports,"{'type': 'Point', 'coordinates': [107.737999, ..."
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports,"{'type': 'Point', 'coordinates': [31.385, 58.6..."
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports,"{'type': 'Point', 'coordinates': [-70.779198, ..."


In [30]:
airports.to_json("output/cleaned_airports.json", orient="records")
airports.to_csv("output/cleaned_airports.csv")

In [9]:
columns = ['Name','Latitude','Longitude','location']
df_airports = pd.DataFrame(columns=columns)

In [11]:
db = MongoClient()
max_distance = 15000 #meters    

for i in cities:
    query = {'location': {'$near': SON([('$geometry', SON([('type', 'Point'), ('coordinates', [i[1], i[0]])])), ('$maxDistance', max_distance)])}}
    for airport in db.company_location.airports.find(query):
        new_row_airport = {'Name':airport['Name'],'Latitude':airport['Latitude'],'Longitude':airport['Longitude'],'location':airport['location']}
        df_airports = df_airports.append(new_row_airport, ignore_index=True)

In [12]:
df_airports.to_csv("output/geo_airports.csv")

# Starbucks

In [14]:
starbucks = pd.read_csv("input/starbucks.csv")
starbucks.dropna(inplace=True)
starbucks["location"] = ''

In [15]:
for index, row in starbucks.iterrows():
    starbucks["location"][index] = Point((row['Longitude'], row['Latitude']))

<ipython-input-15-51ba8cf9e176>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starbucks["location"][index] = Point((row['Longitude'], row['Latitude']))


In [16]:
starbucks

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,location
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,"{'type': 'Point', 'coordinates': [1.53, 42.51]}"
11,Starbucks,1579-122101,HCT Abu Dhabi Women's College Block,Licensed,"Najda Street, Higher Colleges of Technology",Abu Dhabi,AZ,AE,3167,26426280,GMT+04:00 Asia/Dubai,54.37,24.49,"{'type': 'Point', 'coordinates': [54.37, 24.49]}"
12,Starbucks,32595-122105,Standard Chartered Building,Licensed,"Khalidiya St., Beside Union Cooperative Society",Abu Dhabi,AZ,AE,3167,26359275,GMT+04:00 Asia/Muscat,55.69,24.19,"{'type': 'Point', 'coordinates': [55.69, 24.19]}"
20,Starbucks,32767-131566,Shangri-La Souq,Licensed,"Shangri-La Souk, Um Al Nar",Abu Dhabi,AZ,AE,3167,25581641,GMT+04:00 Asia/Dubai,54.51,24.42,"{'type': 'Point', 'coordinates': [54.51, 24.42]}"
45,Starbucks,32640-131563,Tawam Hospital,Licensed,"Al Ain Abu Dhabi Rd, Khalifa Bin Zayed, Al Mak...",Al Ain,AZ,AE,3167,37677581,GMT+04:00 Asia/Muscat,55.65,24.19,"{'type': 'Point', 'coordinates': [55.65, 24.19]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25594,Starbucks,27082-246744,Emart,Licensed,"168 Phan Văn Trị, Quận Gò Vấp, Khách sạn Rex",Thành Phố Hồ Chí Minh,SG,VN,70000,08 3588 0146,GMT+000000 Asia/Saigon,106.69,10.82,"{'type': 'Point', 'coordinates': [106.69, 10.82]}"
25595,Starbucks,21401-212072,Rex,Licensed,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,08 3824 4668,GMT+000000 Asia/Saigon,106.70,10.78,"{'type': 'Point', 'coordinates': [106.7, 10.78]}"
25596,Starbucks,24010-226985,Panorama,Licensed,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,08 5413 8292,GMT+000000 Asia/Saigon,106.71,10.72,"{'type': 'Point', 'coordinates': [106.71, 10.72]}"
25597,Starbucks,47608-253804,Rosebank Mall,Licensed,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,27873500159,GMT+000000 Africa/Johannesburg,28.04,-26.15,"{'type': 'Point', 'coordinates': [28.04, -26.15]}"


In [17]:
starbucks.to_json("output/cleaned_starbucks.json", orient="records")
starbucks.to_csv("output/cleaned_starbucks.csv")

In [18]:
columns = ['Name','Latitude','Longitude','location']
df_starbucks = pd.DataFrame(columns=columns)

In [19]:
db = MongoClient()
max_distance = 10000 #meters    

for i in cities:
    for starbuck in db.company_location.starbucks.find(query):
        query = {'location': {'$near': SON([('$geometry', SON([('type', 'Point'), ('coordinates', [i[1], i[0]])])), ('$maxDistance', max_distance)])}}
        new_row_starbuck = {'Name':starbuck['Brand'],'Latitude':starbuck['Latitude'],'Longitude':starbuck['Longitude'],'location':starbuck['location']}
        df_starbucks = df_starbucks.append(new_row_starbuck, ignore_index=True)

In [20]:
df_starbucks.to_csv("output/geo_starbucks.csv")

## Foursque

In [21]:
def requestPlaces(lat,long,query,categories):
    baseUrl = "https://api.foursquare.com/v2/venues/search"
    client_id = f'?client_id=DZBMMDP2GSE3WPCHFQPFYIMF0DYTRM5JIYQNQWTQSJINYSDI'
    client_secret = f'&client_secret=PVYNMXJ2RZ5SOZHR1L0ECYE2EPRBKHY4DTSLD4Y3SWSQ4JER'
    location = f'&ll={lat},{long}'
    radius = '&radius=1500'
    query = f'&query={query}'
    limit = f'&limit=20'
    version = '&v=20210401'
    categoryId = f'&{categories}'
    
    url = baseUrl+client_id+client_secret+location+query+limit+categoryId+radius+version
 
    res = requests.get(url)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

In [22]:
columns = ['Name','Latitude','Longitude','location']
df_schools = pd.DataFrame(columns=columns)
df_gyms = pd.DataFrame(columns=columns)

In [23]:
for i in cities:
    rq_schools = requestPlaces(i[0],i[1],'school','4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,52e81612bcbc57f1066b7a45,52e81612bcbc57f1066b7a46')
    rq_gyms = requestPlaces(i[0],i[1],'gym','4bf58dd8d48988d175941735')
    for j in rq_schools['response']['venues']:
        new_row_schools = {'Name':j['name'],'Latitude':j['location']['lat'],'Longitude':j['location']['lng'],'location':Point((j['location']['lng'],j['location']['lat']))}
        df_schools = df_schools.append(new_row_schools, ignore_index=True)
    for j in rq_gyms['response']['venues']:
        new_row_gyms = {'Name':j['name'],'Latitude':j['location']['lat'],'Longitude':j['location']['lng'],'location':Point((j['location']['lng'],j['location']['lat']))}
        df_gyms = df_gyms.append(new_row_gyms, ignore_index=True)

In [24]:
df_schools.to_csv("output/geo_schools.csv")
df_gyms.to_csv("output/geo_gyms.csv")

# Map

In [25]:
map_1 = KeplerGl()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [28]:
df_airports = pd.read_csv('output/geo_airports.csv')
map_1.add_data(data=df_airports, name='airports')
df_starbucks = pd.read_csv('output/geo_starbucks.csv')
map_1.add_data(data=df_starbucks, name='starbucks')
df_schools = pd.read_csv('output/geo_schools.csv')
map_1.add_data(data=df_schools, name='schools')
df_gyms = pd.read_csv('output/geo_gyms.csv')
map_1.add_data(data=df_gyms, name='gyms')

In [29]:
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ry4uzw9', 'type': …